# Explore the Gensim implementation

In [1]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
from gensim.models import Word2Vec, KeyedVectors
from gensim.test.utils import datapath

In [4]:
wv = KeyedVectors.load_word2vec_format(datapath("/Users/flint/Data/word2vec/GoogleNews-vectors-negative300.bin"), 
                                       binary=True)

## Similarity

In [ ]:
pairs = [
    ('car', 'minivan'),   # a minivan is a kind of car
    ('car', 'bicycle'),   # still a wheeled vehicle
    ('car', 'airplane'),  # ok, no wheels, but still a vehicle
    ('car', 'cereal'),    # ... and so on
    ('car', 'communism'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

In [ ]:
for x, y in wv.most_similar('car'):
    print(x, y)

## Analogy

FRANCE : PARIS = ITALY : ?

PARIS - FRANCE + ITALY

In [ ]:
wv.most_similar(positive=['King', 'woman'], negative=['man'])

## Not matching

In [ ]:
wv.doesnt_match("school professor apple student".split())

## Mean

In [ ]:
vp = wv['school']
vr = wv['professor']
vx = wv['student']
m = (vp + vr + vx) / 3

In [ ]:
wv.similar_by_vector(m)

In [ ]:
pairs = [
    ('lecturer', 'school'),
    ('lecturer', 'professor'),
    ('lecturer', 'student'),
]
for w1, w2 in pairs:
    print('%r\t%r\t%.2f' % (w1, w2, wv.similarity(w1, w2)))

## Context

In [ ]:
wv.most_similar('buy')

In [ ]:
wv.similarity('buy', 'money')

## Train a custom model

In [ ]:
import gensim.models

In [ ]:
sentences = _ # assume there's one document per line, tokens separated by whitespace
model = gensim.models.Word2Vec(sentences=sentences)

## Exercise: train a model from wordnet

In [ ]:
from nltk.corpus import wordnet as wn
import nltk

In [ ]:
words = ['cat', 'dog', 'bird', 'fish']

In [ ]:
h = lambda s: s.hypernyms()
p = lambda s: s.hyponyms()

def get_pseudo_sentences(word, context=3):
    sentences = []
    for s in wn.synsets(word):
        for lemma in s.lemmas():
            sentences.append([lemma.name(), s.name()])
        for i, j in enumerate(s.closure(h)):
            sentences.append([s.name(), j.name()])
            for lemma in j.lemmas():
                sentences.append([lemma.name(), j.name()])
            if i == context:
                break
        for i, j in enumerate(s.closure(p)):
            sentences.append([j.name(), s.name()])
            for lemma in j.lemmas():
                sentences.append([lemma.name(), j.name()])
            if i == context:
                break
    return sentences

In [ ]:
sentences = []
for w in words:
    sentences += get_pseudo_sentences(w)

In [ ]:
model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
model.wv.most_similar('fish')

## Update an existing model

In [ ]:
import pymongo
import nltk
from string import punctuation
import copy

In [ ]:
MO = Word2Vec.load('/Users/flint/Playground/MeaningSpread/w2v-global.model')

In [ ]:
MO.wv.most_similar('fear')

In [ ]:
db = pymongo.MongoClient()['twitter']['tweets']

In [ ]:
tweets = list(db.find())

In [ ]:
corpus = dict([(tweet['id'], tweet['text']) for tweet in tweets])

In [ ]:
nltk_tokenize = lambda text: [x.lower() for x in nltk.word_tokenize(text) if x not in punctuation]

In [ ]:
data = [nltk_tokenize(y) for x, y in corpus.items()]

In [ ]:
M1 = copy.deepcopy(MO)

In [ ]:
M1.train(data, total_examples=MO.corpus_count, epochs=MO.epochs)

In [ ]:
M1.wv.most_similar('pandemic')